In [ ]:
# execute only if trained on cuda 11.6 // make sure it is done before starting to train
# !pip install pytorch_lightning==1.5

In [1]:
import os; os.chdir("..")

In [2]:
from run import main

%load_ext autoreload
%autoreload 2

In [ ]:
main()

In [ ]:
from project.model import SequenceLearner

In [ ]:
device = "cpu"
checkpoint_path = "checkpoints/model-lnn_units=16-lr=0.01-num_epochs=10-lnn_modified=True-v1.ckpt"

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
trainer = SequenceLearner.load_from_checkpoint(
    checkpoint_path)
model = trainer.model
model.to(device)
model.eval()